In [4]:
import os
from pathlib import Path
import pandas as pd
import json
from epjson.epjson_idf import EpJsonIDF
from epjson.simulate_cluster import SimulateCluster

def run_single_building(
        config_dict, 
        output_dir, 
        resource_dir=Path("tests/resources"), 
        idf_dir=Path("D:/Users/zoelh/Dropbox (MIT)/4.S42 Campus Decarb/Energy Modeling/idfs/umi"), #Path("tests/resources/idfs/")
        eplus_loc=Path("C:\EnergyPlusV22-2-0"), 
        ):

    def replace_shoebox_path(old, new=output_dir):
        new_path = os.path.normpath(new)
        old = os.path.normpath(old)
        to_match = new_path.split(os.sep)[-1]
        idx = old.split("\\").index(to_match)
        p = old.split("\\")
        new_p = new_path
        for i in range(idx + 1, len(p)):
            new_p = os.path.join(new_p, p[i])
        return Path(new_p)
    
    weights_df = pd.read_json(resource_dir / "shoebox-weights.json")
    weights_df = weights_df[weights_df["Building Name"] == config_dict["building_id"]]
    idfs = set(weights_df.ShoeboxPath.to_list())

    ### MUTATIONS ###

    # Get retrofit scenario template data
    features_df = pd.read_csv(resource_dir / f"templates_{config_dict['retrofit_scenario']}.csv")

    with open(resource_dir / "template_features_map.json", "r") as f:
        features_map = json.load(f)
    with open(resource_dir / "schedule_bunches.json", "r") as f:
        schedules = json.load(f)

    for idf in idfs:
        try:
            idf_path = idf_dir / idf
            epjson = EpJsonIDF.from_epjson(idf_path.with_suffix(".epJSON"), eplus_loc=eplus_loc)
            features_series = features_df[(features_df.TYPOLOGY == config_dict["TYPOLOGY"]) & (features_df.AGE_ERA == config_dict["AGE_ERA"])].reset_index(drop=True)
            features_series = features_series.iloc[0]

            cols = set(features_series.index) & set(features_map.keys())

            epjson.replace_features(features_series[cols], features_map)
            epjson.update_schedules(schedules)
            epjson.add([(
                "Output:SQLite",
                {"Output:SQLite 1": {"option_type": "Simple"}}
                )])
            epjson.epjson["ZoneInfiltration:DesignFlowRate"]["CoreInfiltration"].update(
                {"air_changes_per_hour": 0}
            )
            
            new_path = replace_shoebox_path(idf_path).with_suffix(".epJSON")
            epjson.save_as(new_path)
            epjson.save_idf(output_path=new_path.parent) #suffix="_test",

        except Exception as e:
            print(f"FAILED ON {idf}")
            raise e
    
    ### SIMULATION ###
    # Get new paths for updated shoeboxes
    new_idfs = weights_df["ShoeboxPath"].apply(replace_shoebox_path)
    weights_df["ShoeboxPath"] = new_idfs
    
    idf_cluster = SimulateCluster(
        idf_list = set(new_idfs),
        epw = config_dict["epw"],
        weights_df = weights_df,
        building_col = "Building Name",
        eplus_location=eplus_loc
    )
    errors = idf_cluster.parallel_simulate()

    ### POST-PROCESS ###
    r = idf_cluster.fetch_building_results_parallel([config_dict["building_id"]])

    return r, errors

In [5]:
config_dict = {
    "building_id": "1",
    "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
    "retrofit_scenario": "test",
    "schedules_scenaro": "baseline",
    "lab_scenario": "baseline",
    "TYPOLOGY": "Residential",
    "AGE_ERA": 1980,
    "lab_weight": 0.8
}

res, err = run_single_building(config_dict, output_dir = Path(os.getcwd()) / "eplus")

C:\Users\zoelh\AppData\Local\Temp\ipykernel_487680\2820399105.py:61: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  epjson.replace_features(features_series[cols], features_map)
INFO:EPJSON:Setting LatentRecoveryEffectiveness to 0.65
INFO:EPJSON:Setting RecoverySettings to Enthalpy
INFO:EPJSON:Setting OccupancySchedule to 60HrWk
INFO:EPJSON:Setting DHWFlowRate to 3.77e-05
INFO:EPJSON:Setting OASchedule to 30HrWk
INFO:EPJSON:Setting VentilationPerArea to 0.000794118
INFO:EPJSON:Setting EconomizerSettings to DifferentialEnthalpy
INFO:EPJSON:Setting CoolingAvailability to AllOn
INFO:EPJSON:Setting CoolingSetpoint to 23.64705882
INFO:EPJSON:Setting EquipmentPowerDensity to 10.0
INFO:EPJSON:Setting PartitionRValue to 0.227857143
INFO:EPJSON:Setting SlabRValue to 2.591349481
INFO:EPJSON:Setting InfiltrationType to AirChanges/Hour
INFO:EPJSON:Setting DHWSupplyTemp to 65.0
INFO:EPJSON:Setting WindowUValue to 1.5
INFO:EPJSON:Set

INFO:EPJSON:Setting SensibleRecoveryEffectiveness to 0.7
INFO:EPJSON:Updating definition of Output:SQLite
INFO:EPJSON:Building idf for D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group41\SAMPLE3\15\SAMPLE3.epJSON
INFO:EPJSON:Converting D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group41\SAMPLE3\15\SAMPLE3.epJSON to idf
INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.09 15:28
INFO:EPJSON:Converted input file format. Exiting.
INFO:EPJSON:EnergyPlus Run Time=00hr 00min  0.10sec
INFO:EPJSON:Command executed successfully.
C:\Users\zoelh\AppData\Local\Temp\ipykernel_487680\2820399105.py:61: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  epjson.replace_features(features_series[cols], features_map)
INFO:EPJSON:Setting LatentRecoveryEffectiveness to 0.65
INFO:EPJSON:Setting RecoverySettings to Enthalpy
INFO:EPJ

In [3]:
res['1']

,Zone Lights Electricity Energy,Zone Electric Equipment Electricity Energy,Zone Ideal Loads Supply Air Total Heating Energy,Zone Ideal Loads Supply Air Total Cooling Energy,Water Use Equipment Heating Energy,Zone Lights Electricity Energy_norm,Zone Electric Equipment Electricity Energy_norm,Zone Ideal Loads Supply Air Total Heating Energy_norm,Zone Ideal Loads Supply Air Total Cooling Energy_norm,Water Use Equipment Heating Energy_norm,ModelArea,DateTime,BuildingId
0,13.374925,26.749849,802.739530,0.000000,0.000000,0.000997,0.001994,0.059850,0.000000,0.000000,13412.436331,2018-01-01 00:00:00,1
1,13.374925,26.749849,714.596747,0.000000,0.000000,0.000997,0.001994,0.053279,0.000000,0.000000,13412.436331,2018-01-01 01:00:00,1
2,13.374925,26.749849,729.177168,0.000000,0.000000,0.000997,0.001994,0.054366,0.000000,0.000000,13412.436331,2018-01-01 02:00:00,1
3,13.374925,26.749849,739.347157,0.000000,0.000000,0.000997,0.001994,0.055124,0.000000,0.000000,13412.436331,2018-01-01 03:00:00,1
4,13.374925,26.749849,742.573232,0.000000,0.000000,0.000997,0.001994,0.055365,0.000000,0.000000,13412.436331,2018-01-01 04:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,13.374925,26.749849,548.140576,14.362590,887.642601,0.000997,0.001994,0.040868,0.001071,0.066181,13412.436331,2018-12-31 19:00:00,1
8756,13.374925,26.749849,553.132862,10.564365,0.000000,0.000997,0.001994,0.041240,0.000788,0.000000,13412.436331,2018-12-31 20:00:00,1
8757,13.374925,26.749849,568.365632,6.759955,0.000000,0.000997,0.001994,0.042376,0.000504,0.000000,13412.436331,2018-12-31 21:00:00,1
8758,13.374925,26.749849,565.101413,4.790377,0.000000,0.000997,0.001994,0.042133,0.000357,0.000000,13412.436331,2018-12-31 22:00:00,1
